In [1]:
import pandas as pd 

In [2]:
df = pd.read_excel('definition_cleaned.xlsx')
df

,Index,Goal,Target,Indicator,definition
0,1.1.1,Goal 1. End poverty in all its forms everywhere,"1.1 By 2030, eradicate extreme poverty for all...",1.1.1 Proportion of the population living belo...,indicator proportion population international ...
1,1.2.1,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the propo...",1.2.1 Proportion of population living below th...,national poverty rate percentage total populat...
2,1.2.2,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the propo...","1.2.2 Proportion of men, women and children of...",following four series used monitor sdg 122 1 o...
3,1.3.1,Goal 1. End poverty in all its forms everywhere,1.3 Implement nationally appropriate social pr...,1.3.1 Proportion of population covered by soci...,indicator reflects proportion persons effectiv...
4,1.4.1,Goal 1. End poverty in all its forms everywhere,"1.4 By 2030, ensure that all men and women, in...",1.4.1 Proportion of population living in house...,following key concepts defined support indicat...
...,...,...,...,...,...
241,17.17.1,Goal 17. Strengthen the means of implementatio...,"17.17 Encourage and promote effective public, ...",17.17.1 Amount in United States dollars commit...,indicator based wbg data amount united states ...
242,17.18.2,Goal 17. Strengthen the means of implementatio...,17.18.2 Number of countries that have national...,17.18.2 Number of countries that have national...,indicator refers number countries national sta...
243,17.18.3,Goal 17. Strengthen the means of implementatio...,17.18.3 Number of countries with a national st...,17.18.3 Number of countries with a national st...,indicator number countries national statistica...
244,17.19.1,Goal 17. Strengthen the means of implementatio...,"17.19 By 2030, build on existing initiatives t...",17.19.1 Dollar value of all resources made ava...,indicator dollar value resources made availabl...


In [6]:
df = pd.read_excel('definition_filled.xlsx')

In [4]:
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hannahz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#remove \n
df['definition'] = df['definition'].replace(r'\n',' ', regex=True) 
#remove multiple whitespace
df['definition'] = df['definition'].replace('\s+', ' ', regex=True)
#remove white space in the beginning and end
df['definition'] = df['definition'].str.strip()
stop_words_l=stopwords.words('english')
# removing special characters and stop words from the text and lower case (age) - age,  (the - ''
df['definition']=df['definition'].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

#remove white space 
df['definition'] = df['definition'].replace('\s+', ' ', regex=True)

In [9]:
import re
#import spacy

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# import numpy for matrix operation
import numpy as np

# import LDA from sklearn
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
#use tf-idf to vectorize the text
tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(df['definition'])
tf_idf_arr=tfidfvectoriser.transform(df['definition'])

In [11]:
tf_idf_arr

<246x3566 sparse matrix of type '<class 'numpy.float64'>'
	with 11080 stored elements in Compressed Sparse Row format>

In [12]:
# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tfidfvectoriser.get_feature_names()

# get the vocb list
vocab_tf_idf

['abduction',
 'ability',
 'able',
 'abnormal',
 'abnormally',
 'above',
 'abundance',
 'abuse',
 'abuses',
 'acceding',
 'accepted',
 'access',
 'accessed',
 'accessibility',
 'accessible',
 'accession',
 'accessions',
 'accidental',
 'accompanied',
 'accordance',
 'according',
 'account',
 'accountable',
 'accounting',
 'accounts',
 'accrues',
 'accurate',
 'achieve',
 'achieved',
 'achievement',
 'achieving',
 'acidification',
 'acidity',
 'acknowledge',
 'acquiescence',
 'across',
 'act',
 'acting',
 'action',
 'actionable',
 'actions',
 'active',
 'actively',
 'activities',
 'actors',
 'acts',
 'actual',
 'actually',
 'acute',
 'ad',
 'adaptation',
 'adapted',
 'adc',
 'added',
 'addition',
 'additional',
 'additionally',
 'address',
 'addressed',
 'addresses',
 'addressing',
 'adequate',
 'adjusted',
 'administered',
 'administration',
 'administrative',
 'adolescent',
 'adopt',
 'adopted',
 'adoption',
 'adult',
 'adults',
 'advancement',
 'advances',
 'advertised',
 'advocate',

In [13]:
lda_model = LatentDirichletAllocation(n_components = 17, max_iter = 20, random_state = 20)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(tf_idf_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_

In [8]:
topic_words.shape

(17, 4166)

In [9]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 10

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['women' 'water' 'years' 'indicator' 'proportion' 'national' 'percentage'
 'aged' 'public']
Topic 2 ['gni' 'unemployment' 'unemployed' 'force' 'bribe' 'conveys' 'net'
 'official' 'neonatal']
Topic 3 ['wildlife' 'detected' 'illegal' 'trade' 'share' 'fdi' 'spp' 'value'
 'energy']
Topic 4 ['employment' 'climate' 'presents' 'accrues' 'unregulated' 'combat'
 'unreported' 'manufacturing' 'making']
Topic 5 ['exports' 'specific' 'co2' 'seas' 'aid' 'species' 'modern' 'clean'
 'period']
Topic 6 ['education' 'material' 'footprint' 'teacher' 'loss' 'ores' 'student'
 'curricula' 'relation']
Topic 7 ['waste' 'quality' 'cost' 'hazardous' 'females' 'survey' 'grade'
 'employed' 'parties']
Topic 8 ['value' 'unsafe' 'adults' 'energy' 'parliaments' 'seats' 'tcg' 'line'
 'drinking']
Topic 9 ['disaster' 'government' 'risk' 'number' 'died' 'missing' 'directly'
 'went' 'affected']
Topic 10 ['total' 'development' 'population' 'per' 'indicator' 'number' 'land'
 'defined' 'proportion']
Topic 11 ['gdp' 't

In [14]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 10

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['water' 'sanitation' 'gdp' 'co' 'transport' 'personal' 'remittances'
 'public' 'tourism']
Topic 2 ['disbursements' 'donors' 'oda' 'gross' 'human' 'rights' 'individuals'
 'months' 'internet']
Topic 3 ['biodiversity' 'area' 'land' 'material' 'forest' 'areas' 'exports'
 'freshwater' 'footprint']
Topic 4 ['grade' 'expenditure' 'income' 'voting' 'product' 'daily' 'bank'
 'tobacco' 'change']
Topic 5 ['debt' 'mva' 'land' 'commitments' 'iffs' 'clean' 'illicit' 'constant'
 'usd']
Topic 6 ['water' 'population' 'people' 'per' 'members' 'unpaid' 'household'
 'renewable' 'defined']
Topic 7 ['age' 'years' 'aged' 'youth' 'alcohol' 'children' 'recycling' 'health'
 'height']
Topic 8 ['official' 'oda' 'assistance' 'share' 'trade' 'flows' 'wildlife'
 'detected' 'material']
Topic 9 ['fdi' 'surveys' 'export' 'part' 'millions' 'inflows' 'schedules' 'us'
 'respondent']
Topic 10 ['indicator' 'population' 'national' 'proportion' 'countries'
 'development' 'education' 'government' 'years']
Topic 11 ['d

In [10]:
doc_topic = lda_model.transform(tf_idf_arr)  

# iterating over ever value till the end value
for n in range(doc_topic.shape[0]):
    
    # argmax() gives maximum index value
    topic_doc = doc_topic[n].argmax()
    
    # document is n+1  
    print ("Indicator", n+1, " -- Topic:" ,topic_doc+1)

Indicator 1  -- Topic: 1
Indicator 2  -- Topic: 1
Indicator 3  -- Topic: 9
Indicator 4  -- Topic: 1
Indicator 5  -- Topic: 2
Indicator 6  -- Topic: 16
Indicator 7  -- Topic: 9
Indicator 8  -- Topic: 6
Indicator 9  -- Topic: 10
Indicator 10  -- Topic: 9
Indicator 11  -- Topic: 1
Indicator 12  -- Topic: 9
Indicator 13  -- Topic: 12
Indicator 14  -- Topic: 15
Indicator 15  -- Topic: 15
Indicator 16  -- Topic: 16
Indicator 17  -- Topic: 16
Indicator 18  -- Topic: 1
Indicator 19  -- Topic: 16
Indicator 20  -- Topic: 16
Indicator 21  -- Topic: 4
Indicator 22  -- Topic: 1
Indicator 23  -- Topic: 6
Indicator 24  -- Topic: 3
Indicator 25  -- Topic: 10
Indicator 26  -- Topic: 12
Indicator 27  -- Topic: 11
Indicator 28  -- Topic: 9
Indicator 29  -- Topic: 13
Indicator 30  -- Topic: 5
Indicator 31  -- Topic: 2
Indicator 32  -- Topic: 10
Indicator 33  -- Topic: 1
Indicator 34  -- Topic: 14
Indicator 35  -- Topic: 10
Indicator 36  -- Topic: 6
Indicator 37  -- Topic: 9
Indicator 38  -- Topic: 10
Indi

In [ ]:
#use simple count verctorizer

In [14]:
cv_vectorizer = CountVectorizer()
cv_arr = cv_vectorizer.fit_transform(df['definition'])

In [16]:
cv_arr

<246x4166 sparse matrix of type '<class 'numpy.int64'>'
	with 11427 stored elements in Compressed Sparse Row format>

In [17]:
# Creating vocabulary array which will represent all the corpus 
vocab_cv = cv_vectorizer.get_feature_names()

# get the vocb list
vocab_cv

['000',
 '0100',
 '0101',
 '0102',
 '0197',
 '02012',
 '020221',
 '020222',
 '020222ad',
 '02022ed',
 '0205',
 '0208',
 '0210',
 '0211',
 '0219',
 '03',
 '0301',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '100billion',
 '1014',
 '101st',
 '105',
 '1051',
 '107',
 '1071',
 '1072',
 '11',
 '110',
 '11011',
 '110133',
 '110139',
 '1121',
 '114',
 '116',
 '1161',
 '117',
 '1171',
 '11the',
 '12',
 '120',
 '121',
 '121fdrawing',
 '122',
 '126',
 '1281',
 '12a',
 '13',
 '1331',
 '14',
 '140',
 '141',
 '1411',
 '1411a',
 '1411b',
 '142',
 '1421',
 '143',
 '1451',
 '14a',
 '14a1',
 '14c1',
 '15',
 '151',
 '1519',
 '152',
 '1524',
 '1531',
 '1549',
 '1591a',
 '1591b',
 '16',
 '1610',
 '16101',
 '16102',
 '1611',
 '1671',
 '1672',
 '17',
 '1718',
 '17cp8',
 '18',
 '1829',
 '189',
 '18cma1',
 '19',
 '190',
 '196',
 '1992',
 '1994',
 '1995data',
 '1a',
 '1cp18',
 '1land',
 '1oao5',
 '20',
 '200',
 '2001the',
 '2005',
 '2008',
 '2009',
 '2010',
 '20102012',
 '2011',
 '20112020',
 '

In [18]:
# Inside this class LDA: define the components:
lda_model = LatentDirichletAllocation(n_components = 17, max_iter = 20, random_state = 20)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(cv_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_

In [115]:
lda_model.components_

array([[0.05882355, 0.05882391, 0.05883139, ..., 1.64066903, 0.05882353,
        0.05882373],
       [0.05882353, 0.05882353, 0.05882353, ..., 0.05882353, 0.05882353,
        0.05882353],
       [0.05882353, 0.05882353, 0.05882353, ..., 0.05882353, 0.05882353,
        0.05882353],
       ...,
       [0.05882353, 0.05882353, 0.05882353, ..., 0.05882353, 0.05882353,
        0.05882353],
       [0.05882353, 0.16837586, 0.05882353, ..., 0.05882354, 0.05882353,
        0.05882353],
       [0.05882353, 0.05882353, 0.05882353, ..., 0.05882354, 0.05882353,
        0.05882353]])

In [19]:

#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['indicator' 'national' 'countries' 'water']
Topic 2 ['government' 'central' 'budgetary' 'general']
Topic 3 ['public' 'open' 'space' 'spaces']
Topic 4 ['agreement' 'climate' 'violence' 'national']
Topic 5 ['population' 'total' 'indicator' 'alcohol']
Topic 6 ['capital' 'total' 'assets' 'percentage']
Topic 7 ['risk' 'reduction' 'disaster' 'indicators']
Topic 8 ['indicator' 'education' 'genetic' 'resources']
Topic 9 ['level' 'total' 'indicator' 'number']
Topic 10 ['development' 'total' 'oda' 'proportion']
Topic 11 ['national' 'rights' 'indicator' 'human']
Topic 12 ['members' 'defined' 'population' 'proportion']
Topic 13 ['access' 'forest' 'proportion' 'services']
Topic 14 ['number' 'convention' 'indicator' 'national']
Topic 15 ['food' 'year' 'loss' 'agriculture']
Topic 16 ['land' 'indicator' 'rights' 'water']
Topic 17 ['policy' 'development' 'urban' 'subscriptions']


In [ ]:
def clean(doc):
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return normalized

# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus]

In [59]:
from nltk.stem.snowball import SnowballStemmer

In [65]:
stemmer = SnowballStemmer('english')
stemmer.stem('indicator')

'indic'

In [61]:
df['definition']

0      indicator proportion population international ...
1      national poverty rate percentage total populat...
2      following four series used monitor sdg 122 1 o...
3      indicator reflects proportion persons effectiv...
4      following key concepts defined support indicat...
                             ...                        
241    indicator based wbg data amount united states ...
242    indicator refers number countries national sta...
243    indicator number countries national statistica...
244    indicator dollar value resources made availabl...
245    indicator tracks proportion countries conducte...
Name: definition, Length: 246, dtype: object

In [62]:
documents = [" ".join([stemmer.stem(word) for word in sentence.split(" ")]) for sentence in df['definition']]

In [63]:
documents

['indic proport popul intern poverti line defin percentag popul live less $190 day 2011 intern price',
 'nation poverti rate percentag total popul live nation poverti line rural poverti rate percentag rural popul live nation poverti line or case separ rural poverti line use rural poverti line urban poverti rate percentag urban popul live nation poverti line or case separ urban poverti line use urban poverti line',
 'follow four seri use monitor sdg 122 1 offici multidimension poverti headcount sex age popul percentag peopl multidimension poor 2 averag share weight depriv intens total popul averag share weight dimens poor peopl depriv among total popul 3 offici multidimension poverti headcount total household percentag household multidimension poor 4 averag share weight depriv intens total household averag share weight dimens poor peopl depriv among total household last updat may 2021 5 multidimension depriv children popul 18 percentag children simultan depriv multipl dimens wellb',
 'i

In [66]:
tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(documents)
tf_idf_arr=tfidfvectoriser.transform(documents)

In [67]:
# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tfidfvectoriser.get_feature_names()

# get the vocb list
vocab_tf_idf

['000',
 '0100',
 '0101',
 '0102',
 '0197',
 '02012',
 '020221',
 '020222',
 '020222ad',
 '02022ed',
 '0205',
 '0208',
 '0210',
 '0211',
 '0219',
 '03',
 '0301',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '100billion',
 '1014',
 '101st',
 '105',
 '1051',
 '107',
 '1071',
 '1072',
 '11',
 '110',
 '11011',
 '110133',
 '110139',
 '1121',
 '114',
 '116',
 '1161',
 '117',
 '1171',
 '11the',
 '12',
 '120',
 '121',
 '121fdraw',
 '122',
 '126',
 '1281',
 '12a',
 '13',
 '1331',
 '14',
 '140',
 '141',
 '1411',
 '1411a',
 '1411b',
 '142',
 '1421',
 '143',
 '1451',
 '14a',
 '14a1',
 '14c1',
 '15',
 '151',
 '1519',
 '152',
 '1524',
 '1531',
 '1549',
 '1591a',
 '1591b',
 '16',
 '1610',
 '16101',
 '16102',
 '1611',
 '1671',
 '1672',
 '17',
 '1718',
 '17cp8',
 '18',
 '1829',
 '189',
 '18cma1',
 '19',
 '190',
 '196',
 '1992',
 '1994',
 '1995data',
 '1a',
 '1cp18',
 '1land',
 '1oao5',
 '20',
 '200',
 '2001the',
 '2005',
 '2008',
 '2009',
 '2010',
 '20102012',
 '2011',
 '20112020',
 '201

In [68]:
lda_model = LatentDirichletAllocation(n_components = 17, max_iter = 20, random_state = 20)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(tf_idf_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_

In [69]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['oda' 'donor' 'disburs' 'gross']
Topic 2 ['violenc' 'forest' 'girl' 'femal']
Topic 3 ['biodivers' 'valu' 'fuel' 'subind']
Topic 4 ['alcohol' 'poverti' 'expenditur' 'aid']
Topic 5 ['materi' 'account' 'consumpt' 'mva']
Topic 6 ['access' 'public' 'internet' 'fdi']
Topic 7 ['land' 'unsaf' 'agricultur' 'livestock']
Topic 8 ['wildlif' 'detect' 'illeg' 'trade']
Topic 9 ['hazard' 'space' 'privat' 'treat']
Topic 10 ['fisheri' '100000' 'ad' 'traffic']
Topic 11 ['disast' 'risk' 'sendai' 'reduct']
Topic 12 ['death' 'mortal' 'achiev' 'parti']
Topic 13 ['victim' 'previous' 'tobacco' 'poison']
Topic 14 ['human' 'right' 'sea' 'law']
Topic 15 ['unpaid' 'member' 'household' 'famili']
Topic 16 ['indic' 'countri' 'develop' 'popul']
Topic 17 ['person' 'unemploy' 'debt' 'land']
